### Generate boolean mask for LMEs
Running this code takes a few hours, so preferable to do it via the `.py` file : `generate_mask_LMEs.py`.

In [14]:
import geopandas as gpd
from information import *
import xarray as xr
from shapely.geometry import Polygon,Point
import numpy as np
import matplotlib.pyplot as plt

In [15]:
rootdir = '/projects/SOCCOM/datasets/LargeMarineEcos/LME66/'
filename = 'LMEs66.shp'
df = gpd.read_file(rootdir+filename)
grid = xr.open_dataset(ppeDict['rootdir']+ppeDict['gridfile'])
# Reorganize geolon coordinate
grid['geolon'] = grid['geolon'].where(grid['geolon']>-180,grid['geolon']+360)

In [16]:
print(df.head())

   OBJECTID  LME_NUMBER                 LME_NAME GROUPING  ARCTIC USLMES  \
0         1        23.0               Baltic Sea      NaN     NaN    NaN   
1         2        52.0           Sea of Okhotsk      NaN     NaN    NaN   
2         3        22.0                North Sea      NaN     NaN    NaN   
3         4         2.0           Gulf of Alaska      NaN     NaN    Yes   
4         5         9.0  Labrador - Newfoundland   Arctic  Arctic    NaN   

   Shape_Leng  Shape_Area    SUM_GIS_KM  \
0  219.326402   61.950021  3.951472e+05   
1  145.932989  213.796584  1.561297e+06   
2  209.486403  103.965247  6.938472e+05   
3  495.473780  211.045622  1.530668e+06   
4  270.847479  114.597691  9.077923e+05   

                                            geometry  
0  POLYGON ((10.97944 54.38055, 10.98611 54.37805...  
1  POLYGON ((156.70705 51.18858, 156.62592 51.160...  
2  POLYGON ((5.27889 61.98027, 5.24917 61.97388, ...  
3  POLYGON ((-149.41141 61.49638, -149.40311 61.4...  
4  MULTIP

In [9]:
ds = xr.Dataset(coords=grid['geolon'].coords)
gridx,gridy = grid['geolon'].values,grid['geolat'].values
xy_arr = np.vstack((gridx.ravel(), gridy.ravel())).T
# Set up numpy dictionary
inside_dict = {}
for index,row in df.iterrows():
    name = row['LME_NAME']
    inside_dict[name]=np.zeros(shape = len(xy_arr))
Populate np arrays with boolean based on whether point in polygon
for i,xy in enumerate(xy_arr):
    if i%100==0:
        print(str(i)+' of '+str(len(xy_arr)))
    for name,inside in inside_dict.items():
        poly = df[df['LME_NAME']==name]['geometry'].iloc[0]
        inside[i]=poly.contains(Point(xy))

In [ ]:
# Reshape boolean and store in Dataset
for name,inside in inside_dict.items():
    da=xr.DataArray(inside.reshape(gridx.shape).T,dims=ds.dims,coords=ds.coords)
    ds[name]=da

In [ ]:
ds.to_netcdf(ppeDict['datasavedir']+'raw/LME66.ESM4.nc')